In [1]:
import os
os.environ['HADOOP_HOME'] = '/home/ics/roskar/hadoop'
import subprocess

In [2]:
host = 'x09y06'

## put files into hadoop


In [3]:
# first copy a file into a temporary location trimming off the header

In [4]:
import tempfile

In [5]:
data_dir = '/zbox/trove/euclid/2Tlc-final/'

In [49]:
blockids = range(30,32)

In [50]:
import re
get_block_ids = re.compile('blk\.(\d+)\.(\d+)\.(\d+)?')

files = []
for dirname, subdirlist, filelist in os.walk(data_dir):
    try: 
        dirnum = int(os.path.basename(dirname))
        if dirnum in blockids: 
            for f in filelist:
                ids = get_block_ids.findall(f)
                if len(ids) > 0:
                    if all(int(x) in blockids for x in ids[0]):
                        files.append(os.path.join(dirname,f))
    except ValueError:
        pass

In [51]:
for filename in files: 
    with open(filename) as f: 
        data = f.read()
        tf = tempfile.NamedTemporaryFile(delete=False) 
        tf.write(data[62500:])
        tf.close()
        subprocess.check_call(['/home/ics/roskar/hadoop/bin/hdfs', 'dfs', '-put', tf.name, '/2Tlc/%s'%os.path.basename(filename)], shell=False)
        os.remove(tf.name)
        print 'copied ', os.path.basename(filename)

copied  blk.30.30.30i
copied  blk.30.31.31i
copied  blk.30.30.31i
copied  blk.30.31.30i
copied  blk.31.31.31i
copied  blk.31.30.30i
copied  blk.31.31.30i
copied  blk.31.30.31i


In [6]:
import findspark
findspark.init()
import sparkhpc

In [78]:
sc.stop()

In [79]:
sc = sparkhpc.start_spark('spark://x09y06:7078', executor_memory='20000M')

### converting bytes to particles

In [80]:
import time

In [81]:
import numpy as np

pdt_lc = np.dtype([('pos', 'f4', 3),('vel', 'f4', 3)])

In [85]:
for i in [30,31]: 
    for j in [30,31]:
        for k in [30,31]:
            print i,j,k

30 30 30
30 30 31
30 31 30
30 31 31
31 30 30
31 30 31
31 31 30
31 31 31


In [82]:
get_block_ids = re.compile('blk\.(\d+)\.(\d+)\.(\d+)?')
include_file = lambda filename: all([int(blk) in blockids for blk in get_block_ids.findall(filename)[0]])

In [83]:
raw_rdd = (sc.binaryFiles('hdfs://%s:/2Tlc/'%host)
             .map(lambda (path,buf): (path,np.frombuffer(buf,dtype=pdt_lc))))

In [84]:
timein = time.time()
size = raw_rdd.map(lambda (f, arr): len(arr)).sum()*24
print '%f MB/s'%(size/1e6/(time.time()-timein))

185.570565 MB/s


In [94]:
ids = []
for i in blockids:
    for j in blockids: 
        for k in blockids:
            ids.append((i,j,k))

ids_map = {x:i for i,x in enumerate(ids)}

In [95]:
ids_map

{(30, 30, 30): 0,
 (30, 30, 31): 1,
 (30, 31, 30): 2,
 (30, 31, 31): 3,
 (31, 30, 30): 4,
 (31, 30, 31): 5,
 (31, 31, 30): 6,
 (31, 31, 31): 7}

In [102]:
from snakebite.client import Client
c = Client('x09y06')

nparts = {}
for f in c.ls(['/2Tlc']):
    blocks = tuple(map(int, get_block_ids.findall(os.path.basename(f['path']))[0]))
    if all(b in blockids for b in blocks): 
        nparts[ids_map[blocks]] = f['length']/24